In [1]:
# Spearman correlations: SFR_1 vs cytokines (from Excel column BJ onward), split by Group_ID
# Save as: spearman_by_group.py

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
from pathlib import Path

# ==== CONFIG ====
INPUT_PATH = r"/mnt/data/UO1_149.xlsx"   # change if needed
SHEET_NAME = 0                                          # or a sheet name
GROUP_COL  = "Group_ID"
TARGET_COL = "SFR_1"
START_COL_LABEL = "BJ"                                  # Excel column to start cytokine block
OUTPUT_CSV = "spearman_SFR1_cytokines_by_group.csv"
MAKE_HEATMAPS = True                                    # set False to skip plots
HEATMAP_DIR = "spearman_heatmaps"

# ==== HELPERS ====
def excel_col_to_index(label: str) -> int:
    """Excel letters ('A', 'AA', 'BJ', ...) -> 0-based index."""
    label = label.upper()
    n = 0
    for c in label:
        n = n * 26 + (ord(c) - 64)
    return n - 1

# ==== LOAD DATA ====
df = pd.read_excel(INPUT_PATH, sheet_name=SHEET_NAME, engine="openpyxl")

# Basic checks
missing = [c for c in [GROUP_COL, TARGET_COL] if c not in df.columns]
if missing:
    raise ValueError(f"Missing required column(s): {missing}")

# Identify cytokine columns by position (from BJ to the end)
start_idx = excel_col_to_index(START_COL_LABEL)
if start_idx >= len(df.columns):
    raise ValueError(f"START_COL_LABEL={START_COL_LABEL} (index {start_idx}) beyond DataFrame width {len(df.columns)}")

cytokine_cols = list(df.columns[start_idx:])
# Keep only numeric cytokine columns (drop purely non-numeric)
numeric_mask = df[cytokine_cols].apply(pd.to_numeric, errors="coerce").notna().any(axis=0)
cytokine_cols = [c for c in cytokine_cols if numeric_mask[c]]

if not cytokine_cols:
    raise ValueError("No numeric cytokine columns found starting at column BJ.")

# Ensure target is numeric (coerce)
df[TARGET_COL] = pd.to_numeric(df[TARGET_COL], errors="coerce")

# ==== COMPUTE SPEARMAN PER GROUP ====
rows = []
for group, gdf in df.groupby(GROUP_COL, dropna=False):
    # Prepare the vector for SFR_1
    x = pd.to_numeric(gdf[TARGET_COL], errors="coerce")

    # Compute for each cytokine
    for cyto in cytokine_cols:
        y = pd.to_numeric(gdf[cyto], errors="coerce")

        # Pairwise complete cases
        mask = x.notna() & y.notna()
        n = int(mask.sum())
        if n < 3:
            rho, p = np.nan, np.nan
        else:
            rho, p = spearmanr(x[mask], y[mask])

        rows.append({
            "Group_ID": group,
            "Cytokine": cyto,
            "N_pairs": n,
            "Spearman_rho": rho,
            "p_value": p,
        })

res = pd.DataFrame(rows)

# ==== MULTIPLE TESTING (BH-FDR) WITHIN EACH GROUP ====
def add_bh_qvalues(df_group):
    pvals = df_group["p_value"].values
    # Handle groups with all-NaN p's safely
    if np.all(np.isnan(pvals)):
        df_group["q_value_BH"] = np.nan
    else:
        # multipletests expects finite p-values; replace NaN with 1 for correction (will become non-sig)
        p_clean = np.where(np.isnan(pvals), 1.0, pvals)
        _, qvals, _, _ = multipletests(p_clean, alpha=0.05, method="fdr_bh")
        # put NaNs back where original p was NaN
        qvals = np.where(np.isnan(pvals), np.nan, qvals)
        df_group["q_value_BH"] = qvals
    return df_group

res = res.groupby("Group_ID", group_keys=False).apply(add_bh_qvalues)

# Order columns nicely
res = res[["Group_ID", "Cytokine", "N_pairs", "Spearman_rho", "p_value", "q_value_BH"]]

# ==== SAVE ====
res.to_csv(OUTPUT_CSV, index=False)
print(f"Saved results to: {Path(OUTPUT_CSV).resolve()}")

# ==== OPTIONAL: HEATMAPS OF RHO BY GROUP ====
if MAKE_HEATMAPS:
    outdir = Path(HEATMAP_DIR)
    outdir.mkdir(parents=True, exist_ok=True)
    # One heatmap per group (cytokines on y-axis)
    for group, gdf in res.groupby("Group_ID"):
        pivot = gdf.set_index("Cytokine")["Spearman_rho"].to_frame()
        # Simple bar-style "heatmap": matplotlib only, no styles/colors specified per instructions
        plt.figure(figsize=(6, max(3, 0.25 * len(pivot))))
        plt.imshow(pivot.values, aspect="auto")
        plt.yticks(range(len(pivot.index)), pivot.index)
        plt.xticks([0], ["rho"])
        plt.title(f"Spearman rho: {TARGET_COL} vs Cytokines\nGroup: {group}")
        plt.colorbar(label="Spearman rho")
        plt.tight_layout()
        fname = outdir / f"spearman_heatmap_{str(group).replace(' ', '_')}.png"
        plt.savefig(fname, dpi=150)
        plt.close()
        print(f"Saved heatmap: {fname.resolve()}")

# ==== OPTIONAL: TOP HITS PER GROUP (print to console) ====
top = (
    res.dropna(subset=["q_value_BH"])
       .sort_values(["Group_ID", "q_value_BH", "p_value", "Spearman_rho"])
       .groupby("Group_ID")
       .head(10)
)
print("\nTop associations per group (by BH-FDR q, then p):")
print(top.to_string(index=False))


ModuleNotFoundError: No module named 'statsmodels'